In [7]:
import pandas as pd
import numpy as np
from numpy import *
from pyfasta import Fasta

In [42]:
s = "@1:924880-944581W:ENST00000616125:6:2230:99:304:S/2"
s.split(':')[2]

'ENST00000616125'

In [8]:
def readScottsCluster(fn):
    tr_clust = {}
    tr_clust_inv = {}
    
    ft = open(fn)
    cNum = 0
    for line in ft:
        while(line != '\n'):
              tr_clust[line[:-1].lstrip('@')] = cNum
        cNum += 1
    
    for k,v in tr_clust.iteritems():
        if v in tr_clust_inv:
            tr_clust_inv[v].append(k)
        else:
            tr_clust_inv[v] = [k]
    return tr_clust, tr_clust_inv

In [9]:
def readTrueLabels(fn):
    groundTruth_clust = {}
    groundTruth_clust_inv = {}
    ft = open(fn)
    
    for line in ft:
        readId = line[:-1].lstrip('@')
        transcriptId = line.split(':')[2]
        groundTruth_clust[readId] = transcriptId
        for _ in range(3):
            ft.next()
    
    for k,v in groundTruth_clust.iteritems():
        if v in groundTruth_clust_inv:
            groundTruth_clust_inv[v].append(k)
        else:
            groundTruth_clust_inv[v] = [k]
    return groundTruth_clust, groundTruth_clust_inv

In [10]:
import itertools

class Classification:
    TruePos, FalsePos, TrueNeg, FalseNeg = range(4)

def classType(true1, true2, pred1, pred2):
    if true1 == true2:
        if pred1 == pred2:
            return Classification.TruePos
        else: # truely the same, predicted different
            return Classification.FalseNeg
    else: # truly different
        if pred1 == pred2: #predicted same
            return Classification.FalsePos
        else:
            return Classification.TrueNeg

def accuracyExpressed(groundTruth_clust, tr_clust):
    #count true postive for each pair of transcripts O(N^2)
    tp, fp, tn, fn = 0, 0, 0, 0
    for tr_1, tr_2 in itertools.combinations(tr_clust.keys(), 2):
        if tr_1 not in groundTruth_clust or tr_2 not in groundTruth_clust:
            continue
        ct = classType(groundTruth_clust[tr_1], groundTruth_clust[tr_2], tr_clust[tr_1], tr_clust[tr_2]) 
        if ct == Classification.TruePos:
            tp += 1
        elif ct == Classification.TrueNeg:
            tn += 1
        elif ct == Classification.FalsePos:
            fp += 1
        elif ct == Classification.FalseNeg:
            fn += 1
    return tp, fp, tn, fn

        
def accuracyExpressedFast(groundTruth_clust, groundTruth_clust_inv,
                          tr_clust, tr_clust_inv):
    #Hackish and faster way to calculate accuracy
    num = len(set(tr_clust.keys()) & set(groundTruth_clust.keys()))
    tp, fp, tn, fn = 0, 0, 0, 0
    for clustName, clustMems in tr_clust_inv.iteritems():
        for tr_1, tr_2 in itertools.combinations(clustMems,2):
            if tr_1 not in groundTruth_clust or tr_2 not in groundTruth_clust:
                continue
            if groundTruth_clust[tr_1] == groundTruth_clust[tr_2]:
                tp += 1
            else:
                fp += 1
    for clustName, clustMems in groundTruth_clust_inv.iteritems():
        for tr_1, tr_2 in itertools.combinations(clustMems,2):
            if tr_1 not in tr_clust or tr_2 not in tr_clust:
                continue
            if tr_clust[tr_1] != tr_clust[tr_2]:
                fn += 1
    nc2 = (num * (num-1)) / 2
    tn = nc2 - (fp + tp + fn)
    return tp, fp, tn, fn